In [ ]:
import os
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'
print('Device:', DEVICE)

In [ ]:
if os.getcwd() == '/kaggle/working':
    INPUT_DIR = '/kaggle/input'
else:
    INPUT_DIR = './input'

PLANT_DIR = os.path.join(INPUT_DIR, 'plant-pathology-2020-fgvc7')
IMAGE_DIR = os.path.join(PLANT_DIR, 'images')

In [ ]:
train_csv = os.path.join(PLANT_DIR, 'train.csv')
test_csv = os.path.join(PLANT_DIR, 'test.csv')

train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

CLASSES = list(train_df.columns[1:])

print('Classes:', CLASSES)

In [ ]:
def read_image_as_numpy(name):
    path = os.path.join(IMAGE_DIR, name+'.jpg')
    tmp = Image.open(path)
    image = np.array(tmp)
    tmp.close()
    return image

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    H = 1365
    W = 2048
    HCROP = 1344
    WCROP = 2016

    def __init__(self, df, size=None, tflag=False):
        super().__init__()
        self.df = df
        self.size = size
        self.tflag = tflag
        
        classes = df.columns[1:]
        if len(classes) == 0:
            self.labels = None
        else:
            v01 = df[classes].values
            self.labels = (v01 * np.array([0,1,2,3])).sum(axis=1)

        box = []
        if size is not None:
            h, w = size
            assert h <= self.HCROP and w <= self.WCROP
            if tflag:
                box.append(A.RandomCrop(self.HCROP, self.WCROP))
            else:
                box.append(A.CenterCrop(self.HCROP, self.WCROP))
            if h != self.HCROP or w != self.WCROP:
                box.append(A.Resize(h, w))       
        if tflag:
            add = [
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.RandomBrightness(p=0.5),
                A.RandomContrast(p=0.5),
                A.RandomGamma(p=0.5),
            ]
            box.extend(add)
        self.transform_numpy = A.Compose(box)
        box.append(A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)))
        box.append(ToTensorV2())
        self.transform_torch = A.Compose(box)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = self.get_oriented_image(idx)
        image = self.transform_torch(image=image)['image']
        if self.labels is None:
            return image
        else:
            return image, self.labels[idx]

    def get_name(self, idx):
        return self.df.iloc[idx, 0]
    
    def get_oriented_image(self, idx):
        name = self.df.iloc[idx, 0]
        image = read_image_as_numpy(name)
        if image.shape[1] < image.shape[0]:
            image = image.transpose(1, 0, 2)
        return image

    def get(self, idx):
        image = self.get_oriented_image(idx)
        image = self.transform_numpy(image=image)['image']
        return image

In [ ]:
def fetch_from_batch(batch):
    if isinstance(batch, torch.Tensor):
        x, y = batch, None
    else:
        x, y = batch
    return x, y

In [ ]:
def make_features(model, nf, ds):
    dl = torch.utils.data.DataLoader(ds, batch_size=4, shuffle=False)
    model.eval()
    model.to(DEVICE)
    features = np.empty([len(ds), nf])
    beg = 0
    with torch.no_grad():
        for batch in dl:
            x, y = fetch_from_batch(batch)
            x = x.to(DEVICE)
            batch_features = model(x)
            end = beg + len(batch_features)
            features[beg:end] = batch_features
            beg = end
    return features

In [ ]:
class FeatureDataset(torch.utils.data.Dataset):
    def __init__(self, df, filepath):
        super().__init__()
        self.df = df
        self.features = np.loadtxt(filepath, dtype=np.float32)
        
        classes = df.columns[1:]
        if len(classes) == 0:
            self.labels = None
        else:
            v01 = df[classes].values
            self.labels = (v01 * np.array([0,1,2,3])).sum(axis=1)

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        name = self.df.iloc[idx, 0]
        beg = name.index('_') + 1
        num = int(name[beg:])
        x = torch.from_numpy(self.features[num])
        if self.labels is None:
            return x
        else:
            return x, self.labels[idx]

In [ ]:
class Stat:
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.loss_sum = 0
        self.loss_tot = 0
        self.acc_sum = 0
        self.acc_tot = 0
        
    def update(self, pred, y, loss):
        self.loss_sum += loss * len(y)
        self.loss_tot += len(y)

        probs = F.softmax(pred, dim=1)
        _, labels = torch.max(probs, dim=1)
        same = torch.sum(labels==y, dim=0).item()

        self.acc_sum += same
        self.acc_tot += len(y)

    def summary(self):
        loss = self.loss_sum / self.loss_tot if self.loss_tot > 0 else 0.
        acc = self.acc_sum / self.acc_tot if self.acc_tot > 0 else 0.
        return loss, acc

In [ ]:
class Trainer:
    def __init__(self, model, loss_fn, optimizer):
        model.to(DEVICE)

        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer


    def train(self, train_ds, n_epoch):
        self.train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)

        self.stats = []
        for epoch in range(n_epoch):
            train_stat = self.train_epoch()
            self.stats.append((epoch, train_stat))

            
    def train_and_valid(self, train_ds, valid_ds, n_epoch):
        self.train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
        self.valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=64, shuffle=False)

        self.stats = []
        for epoch in range(n_epoch):
            train_stat = self.train_epoch()
            valid_stat = self.valid_epoch()
            self.stats.append((epoch, train_stat, valid_stat))


    def train_epoch(self):
        self.model.train()
        stat = Stat()
        for x, y in self.train_dl:
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            pred = self.model(x)
            loss = self.loss_fn(pred, y)
            loss_item = loss.item()
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            stat.update(pred, y, loss_item)
        return stat

    
    def valid_epoch(self):
        self.model.eval()
        stat = Stat()
        with torch.no_grad():
            for x, y in self.valid_dl:
                x = x.to(DEVICE)
                y = y.to(DEVICE)
                pred = self.model(x)
                loss = self.loss_fn(pred, y)
                loss_item = loss.item()
                stat.update(pred, y, loss_item)
        return stat


    def predict(self, ds):
        self.model.eval()
        dl = torch.utils.data.DataLoader(ds, batch_size=64, shuffle=False)
        preds = np.empty([len(ds),len(CLASSES)], dtype=np.float32)
        preds = torch.from_numpy(preds)
        beg = 0
        with torch.no_grad():
            for batch in dl:
                x, y = fetch_from_batch(batch)
                x = x.to(DEVICE)
                pred = self.model(x)
                if DEVICE != 'cpu':
                    pred = pred.to('cpu')
                end = beg + len(pred)
                preds[beg:end] = pred
                beg = end
        return preds

In [ ]:
def plot_learning(stats, suptitle=None):
    x = []
    t_y = []
    v_y = []
    for e, ts, vs in stats:
        x.append(e)
        t_y.append(ts.summary())
        v_y.append(vs.summary())
    t_y = np.array(t_y)
    v_y = np.array(v_y)

    fig, (ax0, ax1) = plt.subplots(1, 2)

    if suptitle is not None:
        fig.suptitle(suptitle)
    
    ax0.plot(x, t_y[:,0], v_y[:,0])
    ax0.set_xlabel('epoch')
    ax0.set_ylabel('loss')
    txt = 'Valid min: %5.3f at epoch %d' % (v_y[:,0].min(), v_y[:,0].argmin())
    ax0.text(0.4, 0.8, txt, transform=ax0.transAxes)

    ax1.plot(x, t_y[:,1], v_y[:,1])
    ax1.set_xlabel('epoch')
    ax1.set_ylabel('accuracy')
    txt = 'Valid max: %5.3f at epoch %d' % (v_y[:,1].max(), v_y[:,1].argmax())
    ax1.text(0.4, 0.2, txt, transform=ax1.transAxes)

    fig.set_size_inches(24/2.54, 12/2.54)
    fig.tight_layout()

    plt.show()

In [ ]:
def make_learning_curve(size, lr, n_epoch):
    model = nn.Linear(512, len(CLASSES))
    loss_fn = F.cross_entropy
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    trainer = Trainer(model, loss_fn, optimizer)

    path = os.path.join(INPUT_DIR, 'pp-2-features', 'rn18-%d-%d'%size, 'train.txt')
    tds = FeatureDataset(train_df[:1000], path)
    vds = FeatureDataset(train_df[1000:], path)

    trainer.train_and_valid(tds, vds, n_epoch)

    title = 'ResNet18-%d-%d' % size
    plot_learning(trainer.stats, title)

In [ ]:
def make_training(size, lr, n_epoch):
    model = nn.Linear(512, len(CLASSES))
    loss_fn = F.cross_entropy
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    trainer = Trainer(model, loss_fn, optimizer)

    path = os.path.join(INPUT_DIR, 'pp-2-features', 'rn18-%d-%d'%size, 'train.txt')
    tds = FeatureDataset(train_df, path)

    trainer.train(tds, n_epoch)

    return trainer

In [ ]:
def make_submit_file(probs, filename):
    df = test_df.copy()
    df[CLASSES] = probs
    df.to_csv(filename, index=False, float_format='%8.6f')

In [ ]:
make_learning_curve((224,224), 2e-4, 500)

In [ ]:
make_learning_curve((224,336), 5e-4, 500)

In [ ]:
make_learning_curve((336,504), 5e-4, 500)

In [ ]:
make_learning_curve((448,672), 5e-4, 500)

In [ ]:
make_learning_curve((672,1008), 1e-3, 500)

In [ ]:
# Train for submission

In [ ]:
trainer = make_training((448,672), 5e-4, 300)

In [ ]:
size = (448,672)
path = os.path.join(INPUT_DIR, 'pp-2-features', 'rn18-%d-%d'%size, 'test.txt')
ds = FeatureDataset(test_df, path)
preds = trainer.predict(ds)
probs = F.softmax(preds, dim=1)

In [ ]:
make_submit_file(probs, 'submit-rn18-448-672-fe.csv')

In [ ]:
! head -n 3 submit-rn18-448-672-fe.csv
! tail -n 3 submit-rn18-448-672-fe.csv

In [ ]:
# Train for final layer in finetuning

In [ ]:
trainer = make_training((448,672), 5e-4, 50)

In [ ]:
torch.save(trainer.model.state_dict(), 'rn18-448-672-fc.sd')